# LOFAR ionospheric scintillation data tutorial part 2
Maaijke Mevius, June 2023

In this tutorial we will work with fits data. If you have not done so yet, you can download a copy of  [LOFAR_20150302_170000_CS001LBA_LBA_OUTER_Cas-A.fits](https://spaceweather.astron.nl/SolarKSP/data/atdb_process/scintillation_preview/3155/262603/fits_files/LOFAR_20150302_170000_CS001LBA_LBA_OUTER_Cas-A.fits)and [LOFAR_20230503_100000_CS032LBA_LBA_OUTER_.fits](https://spaceweather.astron.nl/SolarKSP/data/atdb_process/scintillation_preview/4603/888136/fits_files/LOFAR_20230503_100000_CS032LBA_LBA_OUTER_.fits).

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import astropy.units as u

You can find some information of handling fits files in the documentation of [astropy.io.fits](https://docs.astropy.org/en/stable/io/fits/index.html). A fits file consists of one or more datasets with headers.

### Excercise:

Open the first data set, inspect the datasets and headers.  Which is the time and which the frequency axis of the `S4_60S` dataset?

In [ ]:
fname = "data/LOFAR_20150302_170000_CS001LBA_LBA_OUTER_Cas-A.fits"
hdu_list = fits.open(fname)

In [ ]:
hdu_list.info()

In [ ]:
hdu_list[1].header

### 
The fits data was generated by processing the *raw* hdf5 data that you worked with in the first part of the tutorial. The data was processed per whole hour, the time resolution has been averaged down to 1 second. For this tutorial we selected the last hour of the observation. The data is stored per station. In this case we took the first station of the list. You can inspect other image and fits data of the same observation at the [LOFAR spaceweather portal](https://spaceweather.astron.nl/SolarKSP/data/atdb_process/scintillation_preview/3155/262603/).

### Excercise:

Plot the power spectrum. This is the first (primary) dataset ("HDU") in the list. For the axis values you can generate them from the `CRVAL`, `CRPIX`, `CDELT` and `NAXIS` parameters in the header. The time format is unix. You can use [astropy.time.Time](https://docs.astropy.org/en/stable/time/index.html) for conversion.

In [ ]:
data = hdu_list[0].data
freqval = hdu_list[0].header["CRVAL2"]
freqidx = hdu_list[0].header["CRPIX2"]
freqstep = hdu_list[0].header["CDELT2"]
nfreq = hdu_list[0].header["NAXIS2"]
freqs = (np.arange(nfreq) - freqidx) * freqstep + freqval

timeval = hdu_list[0].header["CRVAL1"]
timeidx = hdu_list[0].header["CRPIX1"]
timestep = hdu_list[0].header["CDELT1"]
ntime = hdu_list[0].header["NAXIS1"]
times = (np.arange(ntime) - timeidx) * timestep + timeval

In [ ]:
from astropy.time import Time

In [ ]:
times = Time(times, format="unix")

In [ ]:
from matplotlib.dates import ConciseDateFormatter, date2num

In [ ]:
fig, ax = plt.subplots()
ax.imshow(
    data,
    origin="lower",
    interpolation="none",
    extent=[
        date2num(times[0].datetime),
        date2num(times[-1].datetime),
        freqs[0],
        freqs[-1],
    ],
    aspect="auto",
    vmin=0.9,
    vmax=1.1,
)
ax.xaxis_date()
ax.xaxis.set_major_formatter(ConciseDateFormatter(ax.xaxis.get_major_locator()))

### Excercise:

Inspect and plot the other datasets in the file.

### 

Now that you have familiarised yourself with the fits format it is time to look at another interesting event. This time it is processed data from the IDOLS station.

### Excercise:

Open the second data set. Try plotting the power spectrum again. Do you notice anything peculiar in these data?

In [ ]:
fname = "data/LOFAR_20230503_100000_CS032LBA_LBA_OUTER_.fits"
hdu_list = fits.open(fname)

In [ ]:
hdu_list[0].header

In [ ]:
data = hdu_list[0].data
freqval = hdu_list[0].header["CRVAL2"]
freqidx = hdu_list[0].header["CRPIX2"]
freqstep = hdu_list[0].header["CDELT2"]
nfreq = hdu_list[0].header["NAXIS2"]
freqs = (np.arange(nfreq) - freqidx) * freqstep + freqval

timeval = hdu_list[0].header["CRVAL1"]
timeidx = hdu_list[0].header["CRPIX1"]
timestep = hdu_list[0].header["CDELT1"]
ntime = hdu_list[0].header["NAXIS1"]
times = (np.arange(ntime) - timeidx) * timestep + timeval

Unfortuntely there is still a bug in the processing of the IDOLS data. Therefore we have to adjust the frequency and time data. The time is off by 2 hours. The frequency range starts at 10MHz and ends at 80MHz

In [ ]:
times = Time(times, format="unix") + 2 * u.hour  # There is a bug in this data
freqs = np.linspace(10,80,freqs.shape[0])

In [ ]:
fig, ax = plt.subplots()
ax.imshow(
    data,
    origin="lower",
    interpolation="none",
    extent=[
        date2num(times[0].datetime),
        date2num(times[-1].datetime),
        freqs[0],
        freqs[-1],
    ],
    aspect="auto",
    vmin=0.9,
    vmax=1.1,
)
ax.xaxis_date()
ax.xaxis.set_major_formatter(ConciseDateFormatter(ax.xaxis.get_major_locator()))

What could those dark holes in the data be? Let's inspect some external data. In this case look at the [GOES](https://www.ngdc.noaa.gov/stp/satellite/goes-r.html) solar flux data of this specific time. The user guide of this data is [here](https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes16/l2/docs/GOES-R_XRS_L2_Data_Users_Guide.pdf) (PDF).

### Excercise:

Download the GOES 1 minute X-ray flux data. ([**hint**](https://data.ngdc.noaa.gov/platforms/solar-space-observing-satellites/goes/goes18/l2/data/xrsf-l2-avg1m_science/2023/05/sci_xrsf-l2-avg1m_g18_d20230503_v2-2-0.nc))

You can plot the GOES data using [netCDF4](https://unidata.github.io/netcdf4-python/) (which can be installed with `pip install netCDF4`).

In [ ]:
import netCDF4 as nc

In [ ]:
import astropy.units as u

In [ ]:
goes_data = nc.Dataset("data/sci_xrsf-l2-avg1m_g18_d20230503_v2-2-0.nc")

In [ ]:
goes_flux = goes_data.variables["xrsb_flux"][:]
goes_time = Time("2000-01-01T12:00") + goes_data.variables["time"][:] * u.s

In [ ]:
fig, ax = plt.subplots()
ax.plot(goes_time.datetime, goes_flux);
ax.xaxis.set_major_formatter(ConciseDateFormatter(ax.xaxis.get_major_formatter()))

### Excercise:

Overlay the GOES data on top of the LOFAR spectrogram (hint: make a secondary y-axis [using](https://matplotlib.org/stable/gallery/subplots_axes_and_figures/two_scales.html#sphx-glr-gallery-subplots-axes-and-figures-two-scales-py) ax.twinx()).

In [ ]:
fig, ax = plt.subplots()
ax.imshow(
    data,
    origin="lower",
    interpolation="none",
    extent=[
        date2num(times[0].datetime),
        date2num(times[-1].datetime),
        freqs[0],
        freqs[-1],
    ],
    aspect="auto",
    vmin=0.9,
    vmax=1.1,
)
ax.xaxis_date()
ax.xaxis.set_major_formatter(ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax.set_ylabel("Frequency (MHz)")
xlims = ax.get_xlim()
ax2 = ax.twinx()
ax2.plot(goes_time.datetime, goes_flux, color='red', label='GOES flux');
ax2.set_xlim(xlims)
ax2.set_ylabel("GOES flux")
ax2.legend();

What you see is D layer absorption due to the enhancement in electron density at this layer, triggered by the solar flares. Even the RFI gets absorbed, which shows that most of the RFI is indeed caused by man made signals reflected off the ionosphere (at higher altitudes) or signals coming from above, eg. satellites. 